<a href="https://colab.research.google.com/github/nuno-nunes/python-folium/blob/master/Mapa_analise_joinville.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importando arquivos que serão utilizados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Opção para trabalhar offline

In [ ]:
#from google.colab import files
#uploaded = files.upload()
#file = '/content/drive/My Drive/python/bombeiros/db-bombeiros-jlle-jun-20-limpo-legenda - mapa-acidentes-2020 (1).csv'

###Bibliotecas


In [ ]:
import pandas as pd
import folium
import re
import os
import branca as bc
import seaborn as sns
from folium.plugins import BeautifyIcon
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


###Lendo e verificando os dados

In [ ]:
file = '/content/drive/My Drive/python/bombeiros/db-bombeiros-jlle-jun-20-limpo-legenda - mapa-acidentes-2020 (1).csv'
df_import = pd.read_csv(file)
df_import

,Unnamed: 0,data,ano,data2,semana,mes,hora,endereco,descricao,tipo_ocorrência,legenda,cor,bairro,latitude,longitude,endereco_completo
0,0,2020-06-03T20:21:00Z,2020,03/06,23,6,17h18,"Rua Estrada do Oeste, 1400",Queda de Moto,Acidente de Trânsito,Moto,#E55367,Centro,-26.195219,-48.890316,"Rua Estrada do Oeste, 1400, Centro, Joinville,..."
1,1,2020-06-03T18:21:00Z,2020,03/06,23,6,15h18,"Rua Dorothovio do Nascimento, 1130",Queda de Moto,Acidente de Trânsito,Moto,#E55367,Zona Industrial Norte,-26.227476,-48.823582,"Rua Dorothovio do Nascimento, 1130, Zona Indus..."
2,3,2020-06-03T17:35:00Z,2020,03/06,23,6,14h32,"Rua Piratuba, 1133",Carro x Moto,Acidente de Trânsito,Moto x Carro,#DFB235,Saguaçu,-26.267550,-48.837001,"Rua Piratuba, 1133, Saguaçu, Joinville, Santa ..."
3,4,2020-06-03T16:20:00Z,2020,03/06,23,6,13h17,Rua Quinze de Novembro,Carro x Moto,Acidente de Trânsito,Moto x Carro,#DFB235,Nova Brasília,NaN,NaN,"Rua Quinze de Novembro, Nova Brasília, Joinvil..."
4,5,2020-06-03T15:14:00Z,2020,03/06,23,6,12h08,"Rua Sao Firmino, 1296",Moto x Animal,Acidente de Trânsito,Moto,#E55367,Nova Brasília,NaN,NaN,"Rua Sao Firmino, 1296, Nova Brasília, Joinvill..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,2713,2020-01-03T13:28:00Z,2020,03/01,1,1,10h26,"Rua Monsenhor Gercino, 6975",Carro x Moto,Acidente de Trânsito,Moto x Carro,#DFB235,Itaum,-26.330427,-48.837613,"Rua Monsenhor Gercino, 6975, Itaum, Joinville,..."
1060,2716,2020-01-02T23:24:00Z,2020,02/01,1,1,20h13,"Rua Papa Joao XXIII, 541",Carro x Pedestre,Atropelamento,Carro,#E07829,Jardim Iririú,NaN,NaN,"Rua Papa Joao XXIII, 541, Jardim Iririú, Joinv..."
1061,2718,2020-01-02T21:55:00Z,2020,02/01,1,1,18h52,"Rua Uruguai, 250",Queda de Moto,Acidente de Trânsito,Moto,#E55367,Itaum,-26.328205,-48.840306,"Rua Uruguai, 250, Itaum, Joinville, Santa Cata..."
1062,2722,2020-01-02T16:36:00Z,2020,02/01,1,1,13h20,"Rua Tuiuti, 2936",Queda de Moto,Acidente de Trânsito,Moto,#E55367,Aventureiro,-26.239733,-48.818731,"Rua Tuiuti, 2936, Aventureiro, Joinville, Sant..."


###Filtrando os dados

In [ ]:
df = df_import.copy().dropna(axis=0)
df.drop(df.columns[0], axis=1)
#pontos com problema de localização
#2579
#2358
#495

df.rename(columns={ df.columns[0]: "index" }, inplace = True)
df

,index,data,ano,data2,semana,mes,hora,endereco,descricao,tipo_ocorrência,legenda,cor,bairro,latitude,longitude,endereco_completo
0,0,2020-06-03T20:21:00Z,2020,03/06,23,6,17h18,"Rua Estrada do Oeste, 1400",Queda de Moto,Acidente de Trânsito,Moto,#E55367,Centro,-26.195219,-48.890316,"Rua Estrada do Oeste, 1400, Centro, Joinville,..."
1,1,2020-06-03T18:21:00Z,2020,03/06,23,6,15h18,"Rua Dorothovio do Nascimento, 1130",Queda de Moto,Acidente de Trânsito,Moto,#E55367,Zona Industrial Norte,-26.227476,-48.823582,"Rua Dorothovio do Nascimento, 1130, Zona Indus..."
2,3,2020-06-03T17:35:00Z,2020,03/06,23,6,14h32,"Rua Piratuba, 1133",Carro x Moto,Acidente de Trânsito,Moto x Carro,#DFB235,Saguaçu,-26.267550,-48.837001,"Rua Piratuba, 1133, Saguaçu, Joinville, Santa ..."
5,6,2020-06-03T15:12:00Z,2020,03/06,23,6,12h10,"Rua Coronel Francisco Gomes, 669",Carro x Moto,Acidente de Trânsito,Moto x Carro,#DFB235,Bucarein,-26.318960,-48.843276,"Rua Coronel Francisco Gomes, 669, Bucarein, Jo..."
6,9,2020-06-02T22:45:00Z,2020,02/06,23,6,19h38,Rua Dona Francisca,Queda de Moto,Acidente de Trânsito,Moto,#E55367,Pirabeiraba,-26.218614,-48.904094,"Rua Dona Francisca, Pirabeiraba, Joinville, Sa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,2709,2020-01-03T20:32:00Z,2020,03/01,1,1,17h27,"Rua Teresopolis, 254",Carro x Moto,Acidente de Trânsito,Moto x Carro,#DFB235,Guanabara,-26.323202,-48.824019,"Rua Teresopolis, 254, Guanabara, Joinville, Sa..."
1058,2710,2020-01-03T18:28:00Z,2020,03/01,1,1,15h24,"Rua Tuiuti, 712",Queda de Moto,Acidente de Trânsito,Moto,#E55367,Aventureiro,-26.239733,-48.818731,"Rua Tuiuti, 712, Aventureiro, Joinville, Santa..."
1059,2713,2020-01-03T13:28:00Z,2020,03/01,1,1,10h26,"Rua Monsenhor Gercino, 6975",Carro x Moto,Acidente de Trânsito,Moto x Carro,#DFB235,Itaum,-26.330427,-48.837613,"Rua Monsenhor Gercino, 6975, Itaum, Joinville,..."
1061,2718,2020-01-02T21:55:00Z,2020,02/01,1,1,18h52,"Rua Uruguai, 250",Queda de Moto,Acidente de Trânsito,Moto,#E55367,Itaum,-26.328205,-48.840306,"Rua Uruguai, 250, Itaum, Joinville, Santa Cata..."


In [ ]:
#print(df[df.loc[:,"index"] == 495])
df.drop([1011, 927, 207],inplace=True)

#2579 - 1011
#2358 - 927 
#495 - 207

###Inserindo coordenadas geográficas nos dados

###Layout do mapa


In [ ]:
#Cria um mapa com todos os estilos e um seletor para escolher melhor opção

# map
jlle_map_layout = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=11)

# add tiles to map
folium.raster_layers.TileLayer('Open Street Map').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('Stamen Toner').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('Stamen Watercolor').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('CartoDB Positron').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(jlle_map_layout)

# add layer control to show different maps
folium.LayerControl().add_to(jlle_map_layout)

# display map
jlle_map_layout

In [ ]:
#Layout escolhido
jlle_map = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=12)
folium.raster_layers.TileLayer('CartoDB Positron').add_to(jlle_map)


###Análise: mapa dos tipos de acidentes


In [ ]:
print(df.legenda.unique())
print(df.cor.unique())


['Moto' 'Moto x Carro' 'Carro' 'Outros']
['#E55367' '#DFB235' '#E07829' '#8D6D68']


Mapa 1 - Tipos de acidentes

In [ ]:
df['cor'] = df['legenda']

#'Moto' = "#E55367"
#'Moto x Carro'= "#DFB235"
#'Carro' = "#E07829"
#'Outros' = "#8D6D68"

df['cor'] = df.cor.str.replace(r'(Moto$)', 'red')
df['cor'] = df.cor.str.replace(r'(Moto x Carro)', 'purple')
df['cor'] = df.cor.str.replace(r'(Carro$)', 'cadetblue')
df['cor'] = df.cor.str.replace(r'(Outros)', 'darkgreen')

dicionario_cores = {
		"Moto" : "red",
		"Moto x Carro" : 'purple',
		"Carro" : "cadetblue",
		"Outro" : "darkgree"
	}


df

,index,data,ano,data2,semana,mes,hora,endereco,descricao,tipo_ocorrência,legenda,cor,bairro,latitude,longitude,endereco_completo
0,0,2020-06-03T20:21:00Z,2020,03/06,23,6,17h18,"Rua Estrada do Oeste, 1400",Queda de Moto,Acidente de Trânsito,Moto,red,Centro,-26.195219,-48.890316,"Rua Estrada do Oeste, 1400, Centro, Joinville,..."
1,1,2020-06-03T18:21:00Z,2020,03/06,23,6,15h18,"Rua Dorothovio do Nascimento, 1130",Queda de Moto,Acidente de Trânsito,Moto,red,Zona Industrial Norte,-26.227476,-48.823582,"Rua Dorothovio do Nascimento, 1130, Zona Indus..."
2,3,2020-06-03T17:35:00Z,2020,03/06,23,6,14h32,"Rua Piratuba, 1133",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Saguaçu,-26.267550,-48.837001,"Rua Piratuba, 1133, Saguaçu, Joinville, Santa ..."
5,6,2020-06-03T15:12:00Z,2020,03/06,23,6,12h10,"Rua Coronel Francisco Gomes, 669",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Bucarein,-26.318960,-48.843276,"Rua Coronel Francisco Gomes, 669, Bucarein, Jo..."
6,9,2020-06-02T22:45:00Z,2020,02/06,23,6,19h38,Rua Dona Francisca,Queda de Moto,Acidente de Trânsito,Moto,red,Pirabeiraba,-26.218614,-48.904094,"Rua Dona Francisca, Pirabeiraba, Joinville, Sa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,2709,2020-01-03T20:32:00Z,2020,03/01,1,1,17h27,"Rua Teresopolis, 254",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Guanabara,-26.323202,-48.824019,"Rua Teresopolis, 254, Guanabara, Joinville, Sa..."
1058,2710,2020-01-03T18:28:00Z,2020,03/01,1,1,15h24,"Rua Tuiuti, 712",Queda de Moto,Acidente de Trânsito,Moto,red,Aventureiro,-26.239733,-48.818731,"Rua Tuiuti, 712, Aventureiro, Joinville, Santa..."
1059,2713,2020-01-03T13:28:00Z,2020,03/01,1,1,10h26,"Rua Monsenhor Gercino, 6975",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Itaum,-26.330427,-48.837613,"Rua Monsenhor Gercino, 6975, Itaum, Joinville,..."
1061,2718,2020-01-02T21:55:00Z,2020,02/01,1,1,18h52,"Rua Uruguai, 250",Queda de Moto,Acidente de Trânsito,Moto,red,Itaum,-26.328205,-48.840306,"Rua Uruguai, 250, Itaum, Joinville, Santa Cata..."


In [ ]:
#df['cor'] = pd.cut(
#    df['tipo_ocorrência'], 
#    bins=3, 
#    labels=['green', 'blue', 'red'])

In [ ]:
jlle_map = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=12)

folium.raster_layers.TileLayer('CartoDB Positron').add_to(jlle_map)

for _, case in df.iterrows():
  popup_text = 'Tipo: {:} Data: {:} \
  Endereco: {:} Index: {:}'
  popup_text = popup_text.format(case['descricao'],
                                 case['data2'],
                                 case['endereco'],
                                 case['index'])

  folium.Circle(location=[case['latitude'], 
                                case['longitude']],
                                radius=35,
                                color=case['cor'],
                                popup=popup_text,
                                fill=True).add_to(jlle_map)

#folium.CircleMarker.add_to(jlle_map)
#locais = df[['latitude', 'longitude']].values.tolist()


In [ ]:
jlle_map
jlle_map.save('jlle_map.html')

### Analise 2 - acidentes antes em relação a pandemia


Separa por semanas 


In [ ]:
df['semana_cor'] = df['semana'].copy()
df_antes_quarentena = df[df.semana <= 10]
df_durante_quarentena = df[(df["semana"] <=14) & (df["semana"] >=11)]
df_depois_quarentena = df[df.semana >= 15]

df_depois_quarentena

,index,data,ano,data2,semana,mes,hora,endereco,descricao,tipo_ocorrência,legenda,cor,bairro,latitude,longitude,endereco_completo,semana_cor
0,0,2020-06-03T20:21:00Z,2020,03/06,23,6,17h18,"Rua Estrada do Oeste, 1400",Queda de Moto,Acidente de Trânsito,Moto,red,Centro,-26.195219,-48.890316,"Rua Estrada do Oeste, 1400, Centro, Joinville,...",23
1,1,2020-06-03T18:21:00Z,2020,03/06,23,6,15h18,"Rua Dorothovio do Nascimento, 1130",Queda de Moto,Acidente de Trânsito,Moto,red,Zona Industrial Norte,-26.227476,-48.823582,"Rua Dorothovio do Nascimento, 1130, Zona Indus...",23
2,3,2020-06-03T17:35:00Z,2020,03/06,23,6,14h32,"Rua Piratuba, 1133",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Saguaçu,-26.267550,-48.837001,"Rua Piratuba, 1133, Saguaçu, Joinville, Santa ...",23
5,6,2020-06-03T15:12:00Z,2020,03/06,23,6,12h10,"Rua Coronel Francisco Gomes, 669",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Bucarein,-26.318960,-48.843276,"Rua Coronel Francisco Gomes, 669, Bucarein, Jo...",23
6,9,2020-06-02T22:45:00Z,2020,02/06,23,6,19h38,Rua Dona Francisca,Queda de Moto,Acidente de Trânsito,Moto,red,Pirabeiraba,-26.218614,-48.904094,"Rua Dona Francisca, Pirabeiraba, Joinville, Sa...",23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,1095,2020-04-06T15:14:00Z,2020,06/04,15,4,12h12,"Rua Marquês de Olinda, 1059",Moto x Animal,Acidente de Trânsito,Moto,red,América,-26.288103,-48.863902,"Rua Marquês de Olinda, 1059, América, Joinvill...",15
434,1103,2020-04-06T00:33:00Z,2020,05/04,15,4,20h51,Rua Ipiranga,Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Aventureiro,-26.254472,-48.817733,"Rua Ipiranga, Aventureiro, Joinville, Santa Ca...",15
435,1107,2020-04-05T20:12:00Z,2020,05/04,15,4,17h08,Rua Estrada Morro do Meio,Capotamento/Carro,Acidente de Trânsito,Carro,cadetblue,Morro do Meio,-26.333116,-48.927431,"Rua Estrada Morro do Meio, Morro do Meio, Join...",15
436,1113,2020-04-05T15:48:00Z,2020,05/04,15,4,12h46,"Rua Aracaju, 880",Carro x Carro,Acidente de Trânsito,Carro,cadetblue,Santo Antônio,-26.276750,-48.849900,"Rua Aracaju, 880, Santo Antônio, Joinville, Sa...",15


In [ ]:
df['semana_cor'] = df['semana'].copy()

df.loc[569:1062,'semana_cor'] = "cadetblue"

df.loc[441:568,'semana_cor'] = "red"

df.loc[1:437,'semana_cor'] = "orange"

df

,index,data,ano,data2,semana,mes,hora,endereco,descricao,tipo_ocorrência,legenda,cor,bairro,latitude,longitude,endereco_completo,semana_cor
0,0,2020-06-03T20:21:00Z,2020,03/06,23,6,17h18,"Rua Estrada do Oeste, 1400",Queda de Moto,Acidente de Trânsito,Moto,red,Centro,-26.195219,-48.890316,"Rua Estrada do Oeste, 1400, Centro, Joinville,...",23
1,1,2020-06-03T18:21:00Z,2020,03/06,23,6,15h18,"Rua Dorothovio do Nascimento, 1130",Queda de Moto,Acidente de Trânsito,Moto,red,Zona Industrial Norte,-26.227476,-48.823582,"Rua Dorothovio do Nascimento, 1130, Zona Indus...",orange
2,3,2020-06-03T17:35:00Z,2020,03/06,23,6,14h32,"Rua Piratuba, 1133",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Saguaçu,-26.267550,-48.837001,"Rua Piratuba, 1133, Saguaçu, Joinville, Santa ...",orange
5,6,2020-06-03T15:12:00Z,2020,03/06,23,6,12h10,"Rua Coronel Francisco Gomes, 669",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Bucarein,-26.318960,-48.843276,"Rua Coronel Francisco Gomes, 669, Bucarein, Jo...",orange
6,9,2020-06-02T22:45:00Z,2020,02/06,23,6,19h38,Rua Dona Francisca,Queda de Moto,Acidente de Trânsito,Moto,red,Pirabeiraba,-26.218614,-48.904094,"Rua Dona Francisca, Pirabeiraba, Joinville, Sa...",orange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,2709,2020-01-03T20:32:00Z,2020,03/01,1,1,17h27,"Rua Teresopolis, 254",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Guanabara,-26.323202,-48.824019,"Rua Teresopolis, 254, Guanabara, Joinville, Sa...",cadetblue
1058,2710,2020-01-03T18:28:00Z,2020,03/01,1,1,15h24,"Rua Tuiuti, 712",Queda de Moto,Acidente de Trânsito,Moto,red,Aventureiro,-26.239733,-48.818731,"Rua Tuiuti, 712, Aventureiro, Joinville, Santa...",cadetblue
1059,2713,2020-01-03T13:28:00Z,2020,03/01,1,1,10h26,"Rua Monsenhor Gercino, 6975",Carro x Moto,Acidente de Trânsito,Moto x Carro,purple,Itaum,-26.330427,-48.837613,"Rua Monsenhor Gercino, 6975, Itaum, Joinville,...",cadetblue
1061,2718,2020-01-02T21:55:00Z,2020,02/01,1,1,18h52,"Rua Uruguai, 250",Queda de Moto,Acidente de Trânsito,Moto,red,Itaum,-26.328205,-48.840306,"Rua Uruguai, 250, Itaum, Joinville, Santa Cata...",cadetblue


até 09/03 até <=10

até 30/03 11<=>14

a partir de 1/04 >=15

year_2015_data = crime_rates_per_year_data[crime_rates_per_year_data.YEAR == 2015]

In [ ]:
jlle_map_corona = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=12)

folium.raster_layers.TileLayer('CartoDB Positron').add_to(jlle_map_corona)

for _, case in df.iterrows():
  popup_text = 'Tipo: {:} Data: {:} \
  Endereco: {:} Index: {:}'
  popup_text = popup_text.format(case['descricao'],
                                 case['data2'],
                                 case['endereco'],
                                 case['index'])

  folium.Circle(location=[case['latitude'], 
                                case['longitude']],
                                radius=35,
                                color=case['semana_cor'],
                                popup=popup_text,
                                fill=True).add_to(jlle_map_corona)

#Circle().add_to(jlle_map_corona)
#locais = df[['latitude', 'longitude']].values.tolist()
jlle_map_corona
jlle_map_corona.save('jlle_map_corona.html')

### Mapa Calor - Acidentes 


In [ ]:
calor_map = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=12)

folium.raster_layers.TileLayer('CartoDB Positron').add_to(calor_map)

#folium.CircleMarker.add_to(jlle_map)
locais = df[['latitude', 'longitude']].values.tolist()
HeatMap(locais, radius=20).add_to(calor_map)
calor_map
calor_map.save("calor_map.html") 